<h1><center>CS 455/595a: MLP Demo using TensorFlow</center></h1>
<center>Richard S. Stansbury</center>

This notebook applies the ANN techniques for the Titanic Survivors and Boston Housing Prediction models covered in [1] with the [Titanic](https://www.kaggle.com/c/titanic/) and [Boston Housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html) data sets for DT-based classification and regression, respectively.

Several different approaches to model construction are shown ihe demos below

Reference:

[1] Aurelen Geron. *Hands on Machine Learning with Scikit-Learn & TensorFlow* O'Reilley Media Inc, 2017.

[2] Aurelen Geron. "ageron/handson-ml: A series of Jupyter notebooks that walk you through the fundamentals of Machine Learning and Deep Learning in python using Scikit-Learn and TensorFlow." Github.com, online at: https://github.com/ageron/handson-ml [last accessed 2019-03-01]

**Table of Contents**
1. [Titanic Survivor ANN Classifiers](#Titanic-Survivor-Classifier)
 
2. [Boston Housing Cost Ensemble ANN Regressor](#Boston-Housing-Cost-Estimator)

# Titanic Survivor Classifier

## Set up - Imports of libraries and Data Preparation

In [1]:
from matplotlib import pyplot as plt
%matplotlib inline 
import numpy as np
import pandas as pd
import os

# From: https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb    
def reset_graph():
    tf.reset_default_graph() 
    
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "titanic-logs"
logdir = "{}/run-{}/".format(root_logdir, now)

Import the data and apply pipelines to pre-process the data.

In [2]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split

# Read data from input files into Pandas data frames
data_path = os.path.join("datasets","titanic")
train_filename = "train.csv"
test_filename = "test.csv"

def read_csv(data_path, filename):
    joined_path = os.path.join(data_path, filename)
    return pd.read_csv(joined_path)

# Read CSV file into Pandas Dataframes
train_df = read_csv(data_path, train_filename)

# Defining Data Pre-Processing Pipelines
class DataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attributes):
        self.attributes = attributes
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attributes]

class MostFrequentImputer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        self.most_frequent = pd.Series([X[c].value_counts().index[0] for c in X], 
                                       index = X.columns)
        return self
    
    def transform(self, X):
        return X.fillna(self.most_frequent)


numeric_pipe = Pipeline([
        ("Select", DataFrameSelector(["Age", "Fare", "SibSp", "Parch"])), # Selects Fields from dataframe
        ("Imputer", SimpleImputer(strategy="median")),   # Fills in NaN w/ median value for its column
        ("Scaler", StandardScaler()),
    ])

categories_pipe = Pipeline([
        ("Select", DataFrameSelector(["Pclass", "Sex"])), # Selects Fields from dataframe
        ("MostFreqImp", MostFrequentImputer()), # Fill in NaN with most frequent
        ("OneHot", OneHotEncoder(sparse=False, categories='auto')), # Onehot encode
    ])

preprocessing_pipe = FeatureUnion(transformer_list = [
        ("numeric pipeline", numeric_pipe), 
        ("categories pipeline", categories_pipe)
     ]) 

# Process Input Data Using Pipleines
X_data = preprocessing_pipe.fit_transform(train_df)
y_data = train_df["Survived"].values.reshape(-1,1)

# Process the output data.
feature_names = ["Age", "Fare", "SibSp", "Parch", "Class0", "class1","Sex0", "Sex1"]

print(X_data.shape)
print(y_data.shape)

(891, 9)
(891, 1)


Split the data into a training and validation set.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.33)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(596, 9) (596, 1) (295, 9) (295, 1)


Implementation of the TF.Estimator.DNNClassifier (formerly of TFLearn)

In [4]:
# Construction Phase

import tensorflow as tf

reset_graph()

feature_cols = [tf.feature_column.numeric_column("X", shape=[X_data.shape[1]])]

dnn_clf = tf.estimator.DNNClassifier(hidden_units=[20,20], n_classes=2,
                                     feature_columns=feature_cols)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, batch_size=50, num_epochs=400, shuffle=True)
dnn_clf.train(input_fn=train_input_fn)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)

print(X_train.shape)

eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
                                
eval_results


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\richa\\AppData\\Local\\Temp\\tmp4izoed8g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000138A18A2978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-09T20:31:11Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\richa\AppData\Local\Temp\tmp4izoed8g\model.ckpt-4768
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-09-20:31:13
INFO:tensorflow:Saving dict for global step 4768: accuracy = 0.82033896, accuracy_baseline = 0.6, auc = 0.8530833, auc_precision_recall = 0.83617675, average_loss = 0.5096449, global_step = 4768, label/mean = 0.4, loss = 50.11508, precision = 0.7826087, prediction/mean = 0.40462694, recall = 0.7627119
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4768: C:\Users\richa\AppData\Local\Temp\tmp4izoed8g\model.ckpt-4768


{'accuracy': 0.82033896,
 'accuracy_baseline': 0.6,
 'auc': 0.8530833,
 'auc_precision_recall': 0.83617675,
 'average_loss': 0.5096449,
 'label/mean': 0.4,
 'loss': 50.11508,
 'precision': 0.7826087,
 'prediction/mean': 0.40462694,
 'recall': 0.7627119,
 'global_step': 4768}

Now, let's use plain TensorFlow to implement a neural network using the tf.layers.dense class to define fully-connected (dense) layers of RELU and a softmax of the final output.

In [5]:
reset_graph()

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]


learning_rate = 0.1

num_features = X_train.shape[1]
num_instances = X_train.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("Titanic_MLP"):
    hidden1 = tf.layers.dense(X, 20, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, 10, name="Hidden-2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, 5, name="Hidden-3", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden3, 2, name="Survived")
    output = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('Loss', loss)
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 100
batch_size = 50


# Execution
with tf.Session() as sess:
    init.run()
    step=0
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            step+=1
            X_batch = get_batch(X_train, iteration, batch_size)
            y_batch = get_batch(y_train, iteration, batch_size)
            
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch.reshape(y_batch.shape[0])})
            
        # Save trained model at end of epoch
        save_path = saver.save(sess, "./tf_plain_class.chkp")
        
        #Evaluate on final training batch vs. final testing
        acc_train, loss_summary_str, acc_summary_str = sess.run([accuracy, loss_summary, accuracy_summary],feed_dict={X: X_batch, y: y_batch.reshape(y_batch.shape[0])})
        acc_val = accuracy.eval(feed_dict={X:X_test, y: y_test.reshape(y_test.shape[0])})

        # Logging for Tensor Board for each epoch
        file_writer.add_summary(loss_summary_str,step)
        file_writer.add_summary(acc_summary_str,step)
        
        # Print progress made
        print("{}-Train: {} Test:{}".format(epoch,
                                           acc_train,
                                           acc_val)) 
file_writer.close()

Instructions for updating:
Use keras.layers.dense instead.
0-Train: 0.6304348111152649 Test:0.6000000238418579
1-Train: 0.6304348111152649 Test:0.6000000238418579
2-Train: 0.739130437374115 Test:0.7762711644172668
3-Train: 0.739130437374115 Test:0.7864406704902649
4-Train: 0.739130437374115 Test:0.7932203412055969
5-Train: 0.739130437374115 Test:0.800000011920929
6-Train: 0.739130437374115 Test:0.806779682636261
7-Train: 0.739130437374115 Test:0.7966101765632629
8-Train: 0.782608687877655 Test:0.800000011920929
9-Train: 0.782608687877655 Test:0.803389847278595
10-Train: 0.782608687877655 Test:0.800000011920929
11-Train: 0.782608687877655 Test:0.803389847278595
12-Train: 0.804347813129425 Test:0.810169517993927
13-Train: 0.782608687877655 Test:0.8169491291046143
14-Train: 0.804347813129425 Test:0.8135592937469482
15-Train: 0.804347813129425 Test:0.8203389644622803
16-Train: 0.804347813129425 Test:0.8169491291046143
17-Train: 0.804347813129425 Test:0.8237287998199463
18-Train: 0.82608693

In [8]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

with tf.Session() as sess:
    
    saver.restore(sess, "./tf_plain_class.chkp")
    y_predict = output.eval(feed_dict={X:X_test, y: y_test.reshape(y_test.shape[0])})
    
    con_mx = confusion_matrix(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    precision = precision_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    recall = recall_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    print("Precision: {}, \nRecall: {}, \nConf Mat: {}". format(precision, recall, con_mx))

INFO:tensorflow:Restoring parameters from ./tf_plain_class.chkp
Precision: 0.8137254901960784, 
Recall: 0.7033898305084746, 
Conf Mat: [[158  19]
 [ 35  83]]


In [9]:
reset_graph()

encoder = OneHotEncoder(sparse=False, categories='auto')

saver_callback = tf.keras.callbacks.ModelCheckpoint("./keras_model_class.ckpt", 
                                                 save_weights_only=True,
                                                 verbose=1)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape=(9,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer = tf.train.GradientDescentOptimizer(0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

init = tf.global_variables_initializer()

# Execution
with tf.Session() as sess:
    init.run()
    accuracy = model.fit(X_train, y_train, 
              epochs=400, 
              batch_size=100, 
              callbacks = [saver_callback],
              validation_data = (X_test, y_test))

Train on 596 samples, validate on 295 samples
Epoch 1/400
100/596 [====>.........................] - ETA: 2s - loss: 0.7856 - acc: 0.6300
Epoch 00001: saving model to ./keras_model_class.ckpt
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
596/596 [==============================] - 1s 1ms/sample - loss: 0.7181 - acc: 0.6309 - val_loss: 0.6429 - val_acc: 0.6000
Epoch 2/400
100/596 [====>.........................] - ETA: 0s - loss: 0.6084 - acc: 0.6000
Epoch 00002: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 150us/sample - loss: 0.6139 - acc: 0.6510 - val_loss: 0.5838 - val_acc: 0.6915
Epoch 3/400
100/596 [====>.........................] - ETA: 0s - loss: 0.5704 - acc: 0.6900
Epoch 00003: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 167us/sample - loss: 0.5686 - acc: 0.6997 - val_loss: 0.5570 - val_acc: 0.7220
Epoch 4/400


Epoch 30/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3805 - acc: 0.8700
Epoch 00030: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 123us/sample - loss: 0.4317 - acc: 0.8255 - val_loss: 0.4281 - val_acc: 0.8271
Epoch 31/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4908 - acc: 0.7900
Epoch 00031: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 108us/sample - loss: 0.4330 - acc: 0.8188 - val_loss: 0.4267 - val_acc: 0.8169
Epoch 32/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4613 - acc: 0.8100
Epoch 00032: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 182us/sample - loss: 0.4283 - acc: 0.8205 - val_loss: 0.4237 - val_acc: 0.8271
Epoch 33/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3981 - acc: 0.8600
Epoch 00033: saving model to ./keras_model_class.ckpt
596/596 [============================

100/596 [====>.........................] - ETA: 0s - loss: 0.4417 - acc: 0.8000
Epoch 00060: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 152us/sample - loss: 0.4043 - acc: 0.8238 - val_loss: 0.4165 - val_acc: 0.8102
Epoch 61/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4101 - acc: 0.8500
Epoch 00061: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 177us/sample - loss: 0.4051 - acc: 0.8305 - val_loss: 0.4114 - val_acc: 0.8373
Epoch 62/400
100/596 [====>.........................] - ETA: 0s - loss: 0.2929 - acc: 0.8900
Epoch 00062: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 139us/sample - loss: 0.4098 - acc: 0.8322 - val_loss: 0.4498 - val_acc: 0.8237
Epoch 63/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3912 - acc: 0.8400
Epoch 00063: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 150u

100/596 [====>.........................] - ETA: 0s - loss: 0.3351 - acc: 0.8300
Epoch 00090: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 131us/sample - loss: 0.3955 - acc: 0.8289 - val_loss: 0.4163 - val_acc: 0.8271
Epoch 91/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3352 - acc: 0.8900
Epoch 00091: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 119us/sample - loss: 0.3937 - acc: 0.8339 - val_loss: 0.4154 - val_acc: 0.8068
Epoch 92/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3307 - acc: 0.8700
Epoch 00092: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 108us/sample - loss: 0.3945 - acc: 0.8339 - val_loss: 0.4144 - val_acc: 0.8000
Epoch 93/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4052 - acc: 0.8200
Epoch 00093: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 97us

100/596 [====>.........................] - ETA: 0s - loss: 0.3636 - acc: 0.8200
Epoch 00120: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 125us/sample - loss: 0.3850 - acc: 0.8389 - val_loss: 0.4275 - val_acc: 0.8102
Epoch 121/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4537 - acc: 0.8000
Epoch 00121: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 141us/sample - loss: 0.3915 - acc: 0.8339 - val_loss: 0.4156 - val_acc: 0.8068
Epoch 122/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3205 - acc: 0.8700
Epoch 00122: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 155us/sample - loss: 0.3953 - acc: 0.8372 - val_loss: 0.4304 - val_acc: 0.8034
Epoch 123/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3983 - acc: 0.7800
Epoch 00123: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 1

...

100/596 [====>.........................] - ETA: 0s - loss: 0.3095 - acc: 0.8600
Epoch 00270: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 166us/sample - loss: 0.3686 - acc: 0.8490 - val_loss: 0.4350 - val_acc: 0.8136
Epoch 271/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4328 - acc: 0.8100
Epoch 00271: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 135us/sample - loss: 0.3793 - acc: 0.8406 - val_loss: 0.4347 - val_acc: 0.8000
Epoch 272/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3417 - acc: 0.8700
Epoch 00272: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 120us/sample - loss: 0.3584 - acc: 0.8574 - val_loss: 0.4259 - val_acc: 0.8271
Epoch 273/400
100/596 [====>.........................] - ETA: 0s - loss: 0.2579 - acc: 0.8800
Epoch 00273: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 1

100/596 [====>.........................] - ETA: 0s - loss: 0.4811 - acc: 0.8000
Epoch 00300: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 132us/sample - loss: 0.3645 - acc: 0.8624 - val_loss: 0.4364 - val_acc: 0.7966
Epoch 301/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3257 - acc: 0.8500
Epoch 00301: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 173us/sample - loss: 0.3572 - acc: 0.8557 - val_loss: 0.4433 - val_acc: 0.8068
Epoch 302/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3047 - acc: 0.8900
Epoch 00302: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 186us/sample - loss: 0.3625 - acc: 0.8557 - val_loss: 0.4338 - val_acc: 0.8203
Epoch 303/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3026 - acc: 0.9100
Epoch 00303: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 1

100/596 [====>.........................] - ETA: 0s - loss: 0.3695 - acc: 0.8500
Epoch 00330: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 184us/sample - loss: 0.3650 - acc: 0.8523 - val_loss: 0.4485 - val_acc: 0.8068
Epoch 331/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3351 - acc: 0.8500
Epoch 00331: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 130us/sample - loss: 0.3613 - acc: 0.8557 - val_loss: 0.4493 - val_acc: 0.8000
Epoch 332/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4528 - acc: 0.8200
Epoch 00332: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 108us/sample - loss: 0.3701 - acc: 0.8507 - val_loss: 0.4424 - val_acc: 0.7932
Epoch 333/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3910 - acc: 0.8200
Epoch 00333: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 1

100/596 [====>.........................] - ETA: 0s - loss: 0.2745 - acc: 0.9100
Epoch 00360: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 143us/sample - loss: 0.3845 - acc: 0.8490 - val_loss: 0.4516 - val_acc: 0.7898
Epoch 361/400
100/596 [====>.........................] - ETA: 0s - loss: 0.4029 - acc: 0.8300
Epoch 00361: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 114us/sample - loss: 0.3728 - acc: 0.8490 - val_loss: 0.4354 - val_acc: 0.8102
Epoch 362/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3972 - acc: 0.8800
Epoch 00362: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 133us/sample - loss: 0.3559 - acc: 0.8624 - val_loss: 0.4334 - val_acc: 0.8068
Epoch 363/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3379 - acc: 0.8800
Epoch 00363: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 1

100/596 [====>.........................] - ETA: 0s - loss: 0.3787 - acc: 0.8400
Epoch 00390: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 132us/sample - loss: 0.3505 - acc: 0.8574 - val_loss: 0.4572 - val_acc: 0.8000
Epoch 391/400
100/596 [====>.........................] - ETA: 0s - loss: 0.2647 - acc: 0.9200
Epoch 00391: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 152us/sample - loss: 0.3521 - acc: 0.8725 - val_loss: 0.4487 - val_acc: 0.8136
Epoch 392/400
100/596 [====>.........................] - ETA: 0s - loss: 0.2911 - acc: 0.9100
Epoch 00392: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 137us/sample - loss: 0.3510 - acc: 0.8641 - val_loss: 0.4501 - val_acc: 0.8169
Epoch 393/400
100/596 [====>.........................] - ETA: 0s - loss: 0.3965 - acc: 0.8300
Epoch 00393: saving model to ./keras_model_class.ckpt
596/596 [==============================] - 0s 1

In [10]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

with tf.Session() as sess:
    
    model.load_weights("./keras_model_class.ckpt")
    y_predict = model.predict(X_test)
    
    con_mx = confusion_matrix(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    precision = precision_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    recall = recall_score(y_test.reshape(1,-1)[0], np.argmax(y_predict, axis=1))
    print("Precision: {}, \nRecall: {}, \nConf Mat: {}". format(precision, recall, con_mx))

Precision: 0.85, 
Recall: 0.576271186440678, 
Conf Mat: [[165  12]
 [ 50  68]]


# Boston Housing Cost Estimator

Building off the classifier examples above, this section shows ensemble regressors using bagging and random forests.

## Setup

In [11]:
# Load Data Set
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
boston_housing_data = datasets.load_boston()

scaler = MinMaxScaler()
bouston_housing_data_instances = scaler.fit_transform(boston_housing_data.data)
bouston_housing_data_instances.shape


(506, 13)

In [12]:
reset_graph()

train_X, test_X, train_y, test_y = train_test_split(bouston_housing_data_instances,
                                                   boston_housing_data.target,
                                                   test_size=0.20)

def get_batch(X, iter, size):
    return X[(iter*batch_size) : ((iter+1)*batch_size)]

learning_rate = 0.001

num_features = train_X.shape[1]
num_instances = train_y.shape[0]

# Construction
X = tf.placeholder(tf.float32, shape=(None, num_features), name="X")
y = tf.placeholder(tf.float32, shape=(None), name="y")

with tf.name_scope("Boston-MLP"):
    hidden1 = tf.layers.dense(X, 5, name="Hidden-1", activation = tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, 5, name="Hidden-2", activation = tf.nn.relu)
    output = tf.transpose(tf.layers.dense(hidden2, 1, name="Price"))
    
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.square(y-output))
    
with tf.name_scope("train"): 
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 400
batch_size = 10

# Execution
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        
        for iteration in range(num_instances // batch_size + 1):
            X_batch = get_batch(train_X, iteration, batch_size)
            y_batch = get_batch(train_y, iteration, batch_size)   
 
            sess.run(training_op, feed_dict={X: X_batch,
                                            y: y_batch})
            

        mse_train = loss.eval(feed_dict={X: X_batch,
                                            y: y_batch})
        mse_val = loss.eval(feed_dict={X:test_X, y: test_y})
        
        print("{}-Train: {} Test:{}".format(epoch,
                                           mse_train,
                                           mse_val))

Instructions for updating:
Use tf.cast instead.
0-Train: 180.46746826171875 Test:381.29278564453125
1-Train: 19.795461654663086 Test:89.10183715820312
2-Train: 36.182029724121094 Test:51.98948287963867
3-Train: 43.61358642578125 Test:41.324745178222656
4-Train: 40.9240837097168 Test:35.44710159301758
5-Train: 36.229923248291016 Test:31.698347091674805
6-Train: 33.054847717285156 Test:29.738649368286133
7-Train: 29.91683578491211 Test:28.78458023071289
8-Train: 29.410011291503906 Test:28.517553329467773
9-Train: 28.944133758544922 Test:27.967615127563477
10-Train: 28.62889862060547 Test:27.84853172302246
11-Train: 27.477588653564453 Test:26.65339469909668
12-Train: 27.347824096679688 Test:26.886777877807617
13-Train: 26.630828857421875 Test:26.898841857910156
14-Train: 26.088605880737305 Test:27.149477005004883
15-Train: 25.62571144104004 Test:27.24494171142578
16-Train: 25.195283889770508 Test:27.311918258666992
17-Train: 24.3526611328125 Test:27.260770797729492
18-Train: 23.9375152587

152-Train: 1.3062238693237305 Test:18.572673797607422
153-Train: 1.3004568815231323 Test:18.55205535888672
154-Train: 1.303747534751892 Test:18.506303787231445
155-Train: 1.3613874912261963 Test:18.502227783203125
156-Train: 1.3384037017822266 Test:18.355146408081055
157-Train: 1.3421123027801514 Test:18.351213455200195
158-Train: 1.3952418565750122 Test:18.32588768005371
159-Train: 1.3832318782806396 Test:18.26491355895996
160-Train: 1.4068256616592407 Test:18.25588035583496
161-Train: 1.3912745714187622 Test:18.202878952026367
162-Train: 1.387420892715454 Test:18.22077751159668
163-Train: 1.4046244621276855 Test:18.146223068237305
164-Train: 1.4452699422836304 Test:18.065431594848633
165-Train: 1.4196114540100098 Test:18.051183700561523
166-Train: 1.4668424129486084 Test:17.97926902770996
167-Train: 1.4019179344177246 Test:17.981801986694336
168-Train: 1.4599571228027344 Test:17.921825408935547
169-Train: 1.463340401649475 Test:17.878101348876953
170-Train: 1.4615634679794312 Test:17

310-Train: 1.3435509204864502 Test:16.530088424682617
311-Train: 1.5204696655273438 Test:16.51389503479004
312-Train: 1.2946455478668213 Test:16.520626068115234
313-Train: 1.526056170463562 Test:16.559907913208008
314-Train: 1.3416448831558228 Test:16.545738220214844
315-Train: 1.3512108325958252 Test:16.568775177001953
316-Train: 1.3545030355453491 Test:16.564041137695312
317-Train: 1.3506404161453247 Test:16.567607879638672
318-Train: 1.349765658378601 Test:16.560775756835938
319-Train: 1.3719980716705322 Test:16.55293083190918
320-Train: 1.383283019065857 Test:16.54125213623047
321-Train: 1.4283865690231323 Test:16.52013397216797
322-Train: 1.5625708103179932 Test:16.554651260375977
323-Train: 1.3859975337982178 Test:16.545867919921875
324-Train: 1.389974594116211 Test:16.556289672851562
325-Train: 1.3769655227661133 Test:16.575355529785156
326-Train: 1.3902792930603027 Test:16.55312728881836
327-Train: 1.385424017906189 Test:16.557907104492188
328-Train: 1.4009052515029907 Test:16.

From the above example, you will see that the model converges early. 

In [ ]:
reset_graph()


train_X, test_X, train_y, test_y = train_test_split(bouston_housing_data_instances,
                                                   boston_housing_data.target,
                                                   test_size=0.33)


model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu', input_shape=(13,)))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1))


model.compile(optimizer = tf.train.GradientDescentOptimizer(0.001),loss='mean_squared_error',metrics=['mse'])

init = tf.global_variables_initializer()


# Execution
with tf.Session() as sess:
    init.run()
    model.fit(train_X, train_y, epochs=4000, batch_size=200, validation_data = (test_X, test_y))

Train on 339 samples, validate on 167 samples
Epoch 1/4000
339/339 [==============================] - 1s 2ms/sample - loss: 555.1749 - mean_squared_error: 555.1749 - val_loss: 592.7541 - val_mean_squared_error: 592.7541
Epoch 2/4000
339/339 [==============================] - 0s 69us/sample - loss: 529.4108 - mean_squared_error: 529.4108 - val_loss: 558.4852 - val_mean_squared_error: 558.4852
Epoch 3/4000
339/339 [==============================] - 0s 60us/sample - loss: 494.4642 - mean_squared_error: 494.4642 - val_loss: 509.8567 - val_mean_squared_error: 509.8567
Epoch 4/4000
339/339 [==============================] - 0s 56us/sample - loss: 444.4292 - mean_squared_error: 444.4292 - val_loss: 438.8393 - val_mean_squared_error: 438.8393
Epoch 5/4000
339/339 [==============================] - 0s 38us/sample - loss: 372.4542 - mean_squared_error: 372.4542 - val_loss: 339.9891 - val_mean_squared_error: 339.9891
Epoch 6/4000
339/339 [==============================] - 0s 52us/sample - loss: 2

339/339 [==============================] - 0s 53us/sample - loss: 44.3739 - mean_squared_error: 44.3739 - val_loss: 55.8101 - val_mean_squared_error: 55.8101
Epoch 48/4000
339/339 [==============================] - 0s 30us/sample - loss: 43.9380 - mean_squared_error: 43.9380 - val_loss: 55.4463 - val_mean_squared_error: 55.4463
Epoch 49/4000
339/339 [==============================] - 0s 51us/sample - loss: 43.4833 - mean_squared_error: 43.4833 - val_loss: 54.9197 - val_mean_squared_error: 54.9197
Epoch 50/4000
339/339 [==============================] - 0s 46us/sample - loss: 43.2679 - mean_squared_error: 43.2679 - val_loss: 54.9824 - val_mean_squared_error: 54.9824
Epoch 51/4000
339/339 [==============================] - 0s 51us/sample - loss: 42.7372 - mean_squared_error: 42.7372 - val_loss: 54.1578 - val_mean_squared_error: 54.1578
Epoch 52/4000
339/339 [==============================] - 0s 33us/sample - loss: 42.2665 - mean_squared_error: 42.2665 - val_loss: 53.7982 - val_mean_squar

339/339 [==============================] - 0s 32us/sample - loss: 28.2346 - mean_squared_error: 28.2346 - val_loss: 38.8246 - val_mean_squared_error: 38.8246
Epoch 95/4000
339/339 [==============================] - 0s 35us/sample - loss: 27.9781 - mean_squared_error: 27.9781 - val_loss: 38.8021 - val_mean_squared_error: 38.8021
Epoch 96/4000
339/339 [==============================] - 0s 33us/sample - loss: 27.8494 - mean_squared_error: 27.8494 - val_loss: 38.5640 - val_mean_squared_error: 38.5640
Epoch 97/4000
339/339 [==============================] - 0s 37us/sample - loss: 27.5631 - mean_squared_error: 27.5631 - val_loss: 37.8880 - val_mean_squared_error: 37.8880
Epoch 98/4000
339/339 [==============================] - 0s 41us/sample - loss: 27.7513 - mean_squared_error: 27.7513 - val_loss: 37.6320 - val_mean_squared_error: 37.6320
Epoch 99/4000
339/339 [==============================] - 0s 39us/sample - loss: 27.3008 - mean_squared_error: 27.3008 - val_loss: 37.4427 - val_mean_squar

339/339 [==============================] - 0s 27us/sample - loss: 21.4549 - mean_squared_error: 21.4549 - val_loss: 30.3805 - val_mean_squared_error: 30.3805
Epoch 142/4000
339/339 [==============================] - 0s 34us/sample - loss: 22.2479 - mean_squared_error: 22.2479 - val_loss: 30.3658 - val_mean_squared_error: 30.3658
Epoch 143/4000
339/339 [==============================] - 0s 31us/sample - loss: 21.3957 - mean_squared_error: 21.3957 - val_loss: 30.5610 - val_mean_squared_error: 30.5610
Epoch 144/4000
339/339 [==============================] - 0s 32us/sample - loss: 21.2403 - mean_squared_error: 21.2403 - val_loss: 30.9435 - val_mean_squared_error: 30.9435
Epoch 145/4000
339/339 [==============================] - 0s 30us/sample - loss: 21.1160 - mean_squared_error: 21.1160 - val_loss: 30.3253 - val_mean_squared_error: 30.3253
Epoch 146/4000
339/339 [==============================] - 0s 31us/sample - loss: 21.0197 - mean_squared_error: 21.0197 - val_loss: 30.7168 - val_mean_

339/339 [==============================] - 0s 23us/sample - loss: 19.0083 - mean_squared_error: 19.0083 - val_loss: 27.8215 - val_mean_squared_error: 27.8215
Epoch 189/4000
339/339 [==============================] - 0s 27us/sample - loss: 19.1876 - mean_squared_error: 19.1876 - val_loss: 28.1846 - val_mean_squared_error: 28.1846
Epoch 190/4000
339/339 [==============================] - 0s 59us/sample - loss: 19.0297 - mean_squared_error: 19.0297 - val_loss: 27.2400 - val_mean_squared_error: 27.2400
Epoch 191/4000
339/339 [==============================] - 0s 25us/sample - loss: 19.1127 - mean_squared_error: 19.1127 - val_loss: 27.3504 - val_mean_squared_error: 27.3504
Epoch 192/4000
339/339 [==============================] - 0s 35us/sample - loss: 18.9763 - mean_squared_error: 18.9763 - val_loss: 28.0549 - val_mean_squared_error: 28.0549
Epoch 193/4000
339/339 [==============================] - 0s 33us/sample - loss: 19.0815 - mean_squared_error: 19.0815 - val_loss: 27.6060 - val_mean_

...

339/339 [==============================] - 0s 36us/sample - loss: 12.6291 - mean_squared_error: 12.6291 - val_loss: 14.3356 - val_mean_squared_error: 14.3356
Epoch 2674/4000
339/339 [==============================] - 0s 32us/sample - loss: 12.6928 - mean_squared_error: 12.6928 - val_loss: 13.5935 - val_mean_squared_error: 13.5935
Epoch 2675/4000
339/339 [==============================] - 0s 29us/sample - loss: 12.9435 - mean_squared_error: 12.9435 - val_loss: 14.1713 - val_mean_squared_error: 14.1713
Epoch 2676/4000
339/339 [==============================] - 0s 29us/sample - loss: 12.7647 - mean_squared_error: 12.7647 - val_loss: 14.5915 - val_mean_squared_error: 14.5915
Epoch 2677/4000
339/339 [==============================] - 0s 29us/sample - loss: 12.6633 - mean_squared_error: 12.6633 - val_loss: 13.6026 - val_mean_squared_error: 13.6026
Epoch 2678/4000
339/339 [==============================] - 0s 27us/sample - loss: 12.7060 - mean_squared_error: 12.7060 - val_loss: 13.5718 - val_

339/339 [==============================] - 0s 36us/sample - loss: 12.9239 - mean_squared_error: 12.9239 - val_loss: 13.7101 - val_mean_squared_error: 13.7101
Epoch 2721/4000
339/339 [==============================] - 0s 31us/sample - loss: 12.4728 - mean_squared_error: 12.4728 - val_loss: 13.5731 - val_mean_squared_error: 13.5731
Epoch 2722/4000
339/339 [==============================] - 0s 27us/sample - loss: 12.6227 - mean_squared_error: 12.6227 - val_loss: 13.7233 - val_mean_squared_error: 13.7233
Epoch 2723/4000
339/339 [==============================] - 0s 26us/sample - loss: 12.7890 - mean_squared_error: 12.7890 - val_loss: 13.5672 - val_mean_squared_error: 13.5672
Epoch 2724/4000
339/339 [==============================] - 0s 26us/sample - loss: 12.8012 - mean_squared_error: 12.8012 - val_loss: 14.9521 - val_mean_squared_error: 14.9521
Epoch 2725/4000
339/339 [==============================] - 0s 39us/sample - loss: 14.1862 - mean_squared_error: 14.1862 - val_loss: 15.9646 - val_

339/339 [==============================] - 0s 34us/sample - loss: 12.5218 - mean_squared_error: 12.5218 - val_loss: 13.5807 - val_mean_squared_error: 13.5807
Epoch 2768/4000
339/339 [==============================] - 0s 41us/sample - loss: 12.4096 - mean_squared_error: 12.4096 - val_loss: 13.5879 - val_mean_squared_error: 13.5879
Epoch 2769/4000
339/339 [==============================] - 0s 39us/sample - loss: 12.4217 - mean_squared_error: 12.4217 - val_loss: 13.5496 - val_mean_squared_error: 13.5496
Epoch 2770/4000
339/339 [==============================] - 0s 44us/sample - loss: 12.3936 - mean_squared_error: 12.3936 - val_loss: 13.8300 - val_mean_squared_error: 13.8300
Epoch 2771/4000
339/339 [==============================] - 0s 44us/sample - loss: 12.5375 - mean_squared_error: 12.5375 - val_loss: 13.7959 - val_mean_squared_error: 13.7959
Epoch 2772/4000
339/339 [==============================] - 0s 40us/sample - loss: 12.4372 - mean_squared_error: 12.4372 - val_loss: 13.5768 - val_

339/339 [==============================] - 0s 30us/sample - loss: 12.4217 - mean_squared_error: 12.4217 - val_loss: 13.4759 - val_mean_squared_error: 13.4759
Epoch 2815/4000
339/339 [==============================] - 0s 45us/sample - loss: 12.3961 - mean_squared_error: 12.3961 - val_loss: 13.9865 - val_mean_squared_error: 13.9865
Epoch 2816/4000
339/339 [==============================] - 0s 40us/sample - loss: 13.6010 - mean_squared_error: 13.6010 - val_loss: 16.1882 - val_mean_squared_error: 16.1882
Epoch 2817/4000
339/339 [==============================] - 0s 33us/sample - loss: 13.5155 - mean_squared_error: 13.5155 - val_loss: 13.7508 - val_mean_squared_error: 13.7508
Epoch 2818/4000
339/339 [==============================] - 0s 37us/sample - loss: 12.4811 - mean_squared_error: 12.4811 - val_loss: 13.8923 - val_mean_squared_error: 13.8923
Epoch 2819/4000
339/339 [==============================] - 0s 45us/sample - loss: 13.1928 - mean_squared_error: 13.1928 - val_loss: 16.0163 - val_

339/339 [==============================] - 0s 34us/sample - loss: 12.4801 - mean_squared_error: 12.4801 - val_loss: 13.6706 - val_mean_squared_error: 13.6706
Epoch 2862/4000
339/339 [==============================] - 0s 32us/sample - loss: 12.5378 - mean_squared_error: 12.5378 - val_loss: 13.6346 - val_mean_squared_error: 13.6346
Epoch 2863/4000
339/339 [==============================] - 0s 31us/sample - loss: 12.3795 - mean_squared_error: 12.3795 - val_loss: 13.6933 - val_mean_squared_error: 13.6933
Epoch 2864/4000
339/339 [==============================] - 0s 27us/sample - loss: 12.3388 - mean_squared_error: 12.3388 - val_loss: 13.4280 - val_mean_squared_error: 13.4280
Epoch 2865/4000
339/339 [==============================] - 0s 29us/sample - loss: 12.2900 - mean_squared_error: 12.2900 - val_loss: 13.3830 - val_mean_squared_error: 13.3830
Epoch 2866/4000
339/339 [==============================] - 0s 28us/sample - loss: 12.3102 - mean_squared_error: 12.3102 - val_loss: 13.4287 - val_